In [7]:
import utils
import numpy as np

In [59]:
def region_growing(im: np.ndarray, seed_points: list, T: int) -> np.ndarray:
    """
        A region growing algorithm that segments an image into 1 or 0 (True or False).
        Finds candidate pixels with a Moore-neighborhood (8-connectedness). 
        Uses pixel intensity thresholding with the threshold T as the homogeneity criteria.
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
            seed_points: list of list containing seed points (row, col). Ex:
                [[row1, col1], [row2, col2], ...]
            T: integer value defining the threshold to used for the homogeneity criteria.
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    ### START YOUR CODE HERE ### (You can change anything inside this block)
    # You can also define other helper functions
    segmented = np.zeros_like(im).astype(bool)
    im = im.astype(float)
    
    #Helper function which allows lambda function to accept a tuple and unpack it (makes the functions more readable)
    unpack = lambda f: (lambda args: f(*args))
    
    #Test that pixel coordinates are not outside of the image
    is_in_bounds = unpack(lambda x, y: not (x < 0 or y < 0 or x >= im.shape[0] or y >= im.shape[1]))
    
    #Get (8-connected) neigbors
    get_8_connected = unpack(lambda x, y: [(x-1, y+1),(x, y+1),(x+1, y+1),
                                           (x-1, y  ),         (x+1, y  ),
                                           (x-1, y-1),(x, y-1),(x+1, y-1)])
    
    #Check that pixel is not already explored
    not_explored = unpack(lambda x, y: not segmented[x, y])
    
    for sx, sy in seed_points:
        #Test if pixel belongs to segment by threshold
        test_threshold = unpack(lambda x, y: np.abs(im[sx, sy] - im[x, y]) < T)
        
        frontier = []
        
        #Check if pixel is already in frontier
        not_in_frontier = unpack(lambda x, y: (x, y) not in frontier)
        
        frontier.append((sx, sy))
        
        while len(frontier) > 0:
            x, y = frontier.pop(0)
            segmented[x, y] = True
            neighbors = get_8_connected((x, y))
            neighbors = filter(is_in_bounds, neighbors)
            neighbors = filter(not_explored, neighbors)
            neighbors = filter(not_in_frontier, neighbors)
            neighbors = filter(test_threshold, neighbors)
            frontier.extend(neighbors)
            
        
    return segmented
    ### END YOUR CODE HERE ###

In [62]:
if __name__ == "__main__":
    # DO NOT CHANGE
    im = utils.read_image("defective-weld.png")

    seed_points = [ # (row, column)
        [254, 138], # Seed point 1
        [253, 296], # Seed point 2
        [233, 436], # Seed point 3
        [232, 417], # Seed point 4
    ]
    intensity_threshold = 50
    segmented_image = region_growing(im, seed_points, intensity_threshold)

    assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
        im.shape, segmented_image.shape)
    assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
            segmented_image.dtype)

    segmented_image = utils.to_uint8(segmented_image)
    utils.save_im("defective-weld-segmented.png", segmented_image)

Reading image: images\defective-weld.png
Saving image to: image_processed\defective-weld-segmented.png
